<a href="https://colab.research.google.com/github/gitdiren/Anglecalculator/blob/main/flower%20classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
import os
from torch.optim.lr_scheduler import StepLR
import time
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
drive.mount('/content/drive')
import zipfile



flower_photos = zipfile.ZipFile('/content/drive/MyDrive/flower_photos 2.zip', 'r')
flower_photos.extractall('/tmp')
flower_photos.close()

train_datagen = ImageDataGenerator(rescale = 1./255)

train_set = train_datagen.flow_from_directory('/tmp/flower_photos',
                                               class_mode='categorical',
                                               classes = ['daisy','dandelion','roses','sunflowers','tulips'],
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 color_mode ='grayscale')

train_set.class_indices


torch.manual_seed(42)

# Setting up a Device to decide whether to use GPU(when available) or CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Loading the ResNet-50 model pre-trained on ImageNet
model = models.resnet50(pretrained=True)
num_of_features = model.fc.in_features

# Freezing the earlier layers
for param in model.parameters():
    param.requires_grad = False

# Replacing the final fully connected layer with a new one (for transfer learning)
num_of_classes = 5  # My dataset has 5 classes: daisy,roses,dandelion,sunflowers and tulips
model.fc = torch.nn.Linear(num_of_features, num_of_classes)

# Moving the model to the appropriate device
model.to(device)

# Data Transformations (adding more augmentations)
transform = transforms.Compose([
    transforms.Resize((224, 224)),#resize image to 224x224 pixels
    transforms.RandomHorizontalFlip(), #flips images horizontally, randomly
    transforms.RandomRotation(10),#rotates images by up to 10 degrees randomly
    transforms.ToTensor(),#images are coverted to pytorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #Normalizing images using the mean and standard deviation
])

# Loading Data
data_dir = os.path.expanduser('/tmp/flower_photos')  # Path to the dataset used (images file) on desktop
dataset = ImageFolder(root=data_dir, transform=transform) #loding the images with the requested transformations

# Split dataset into training, validation, and testing sets (split ratios: training:70%, validation:15% and testing:15%)
train_size70 = int(0.7 * len(dataset))
val_size15 = int(0.15 * len(dataset))
test_size15 = len(dataset) - train_size70 - val_size15
train_datast, val_datast, test_datast = random_split(dataset, [train_size70, val_size15, test_size15])

# Defining DataLoaders
train_loader = DataLoader(train_datast, batch_size=32, shuffle=True) #each batch has 32 images and images are shuffled after each epoch
val_loader = DataLoader(val_datast, batch_size=32, shuffle=False)
test_loader = DataLoader(test_datast, batch_size=32, shuffle=False)

# Defining the loss function
loss_function = torch.nn.CrossEntropyLoss() #for classification tasks cross-entropy-loss is preferred

#Hyperparameter tuning: Different learning rates with be iterated
learning_rates = [0.1, 0.01, 0.001]
best_validation_accuracy = 0
best_lrate = None

for lr in learning_rates:
    print(f"Training with learning rate: {lr}")

    # Initializing optimizer as per the current learning rate
    optimizer = torch.optim.Adam(model.fc.parameters(), lr=lr)
    scheduler = StepLR(optimizer, step_size=3, gamma=0.1)  # Automatically decreases the learning rate by a factor(*) of 0.1 every 3 epochs

    num_epochs = 10
    for epoch in range(num_epochs):
        start_time = time.time() # records the time each epoch takes place as system is very slow with cpu
        model.train() #model is set to training mode
        running_loss = 0.0
        for x, y in train_loader: # x: batch of images, y= their corresponding labels
            x, y = x.to(device), y.to(device) #moves x and y to the selected device

            optimizer.zero_grad()  # clears the old gradients from the previous steps, makes zero.
            y_pred = model(x)  # Forward propagation of inputs to get
            loss = loss_function(y_pred, y)  # Function that calculates the loss
            loss.backward()  # Back propagation
            optimizer.step()  # Adjusting the weights

            running_loss += loss.item() * x.size(0)#accumulates the ruuning loss

        scheduler.step()  # Updating the learning rate as per scheduler

        epochloss = running_loss / len(train_loader.dataset) #calculating the average opech loss
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epochloss:.4f}")

        # Validation phase
        model.eval() #sets the model to evaluation mode
        validation_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad(): #disabling the gradient calculation
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)#moaves data to device
                y_pred = model(x)  # Forward propagation
                loss = loss_function(y_pred, y)  # Function that calculates the loss
                validation_loss += loss.item() * x.size(0) #accumulating validation losses

                _, predicted = torch.max(y_pred, 1) # calculating the no of correct predictions
                total += y.size(0) #accumulating the total number of images processed
                correct += (predicted == y).sum().item() #accumulating the total number of correct predictions

        validation_loss = validation_loss / len(val_loader.dataset) #calculating the average validation loss
        validation_accuracy = correct / total
        print(f"Validation Loss: {validation_loss:.4f}, Validation Accuracy: {validation_accuracy:.4f}, Time: {time.time()-start_time:.2f}s")

        # Checking if this is the best validation accuracy yet
        if validation_accuracy > best_validation_accuracy:
            best_validation_accuracy = validation_accuracy
            best_lrate = lr

print(f"Best learning rate: {best_lrate}, with Validation Accuracy: {best_validation_accuracy:.4f}")

# From hyperparameter tuning section best learning rate is discovered as 0.01, final training will be made with
#this rate which will help optimize best parameters and model to converge fully,
#no of epocs could be increased to get a better validation loss, this hasn't been done in this example due to time)


optimizer = torch.optim.Adam(model.fc.parameters(), lr=best_lrate)
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()  # zeroing the gradients
        y_pred = model(x)   # Forward propagation
        loss = loss_function(y_pred, y) # Function that calculates the loss
        loss.backward()  # Back propagation
        optimizer.step()  # Adjusting the weights

        running_loss += loss.item() * x.size(0)



    epoch_loss = running_loss / len(train_loader.dataset) #calculating average epoch loss
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}") #printing average epoch loss

# Testing the model
model.eval() #setting the model to evaluation mode
testing_loss = 0.0
correct = 0
total = 0
with torch.no_grad(): #stopping the gradient calculations
    for x, y in test_loader:#looping through test data
        x, y = x.to(device), y.to(device) #moving data to device
        y_pred = model(x)  # Forward propagation
        loss = loss_function(y_pred, y) # Function that calculates the loss
        testing_loss += loss.item() * x.size(0) #accumulating testing loss

        _, predicted = torch.max(y_pred, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()

testing_loss = testing_loss / len(test_loader.dataset)
test_accuracy = correct / total
print(f"Test Loss is: {testing_loss:.4f}, Test Accuracy is: {test_accuracy:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'resnet50_transfer_learning_best.pth')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 3571 images belonging to 5 classes.
Using device: cuda
Training with learning rate: 0.1
Epoch 1/10, Loss: 17.5381
Validation Loss: 4.7616, Validation Accuracy: 0.7944, Time: 23.77s
Epoch 2/10, Loss: 4.3223
Validation Loss: 2.8057, Validation Accuracy: 0.8598, Time: 21.91s
Epoch 3/10, Loss: 5.4402
Validation Loss: 9.2282, Validation Accuracy: 0.7551, Time: 20.78s
Epoch 4/10, Loss: 2.7059
Validation Loss: 2.4463, Validation Accuracy: 0.8654, Time: 22.76s
Epoch 5/10, Loss: 1.8991
Validation Loss: 2.9704, Validation Accuracy: 0.8673, Time: 21.37s
Epoch 6/10, Loss: 1.8082
Validation Loss: 2.6175, Validation Accuracy: 0.8785, Time: 21.20s
Epoch 7/10, Loss: 1.6008
Validation Loss: 2.5910, Validation Accuracy: 0.8710, Time: 21.58s
Epoch 8/10, Loss: 1.4446
Validation Loss: 2.6854, Validation Accuracy: 0.8710, Time: 21.42s
Epoch 9/10, Loss: 1.3622
Validation Loss